# Import libraries

In [3]:
# General libraries
import numpy as np
from pyDOE import lhs

# Simulation libraries
import skysurvey
import skysurvey_sniapop
import ztfidr.simulation as sim

# Module functions
import ximinf.selection_effects as se

# Plot libraries
import matplotlib.pyplot as plt

# Generate one test sample

In [4]:
brokenalpha_model = skysurvey_sniapop.brokenalpha_model

N = 3000
# alpha_low = 0.271
# alpha_high = 0.083
alpha = 0.161
beta = 3.05 #3.31
mabs = -19.3

# Model the SNe Ia
snia = skysurvey.SNeIa.from_draw(
    size=N,
    zmax=0.2,
    model=brokenalpha_model,
    magabs={
        "x1": "@x1",
        "c": "@c",
        "mabs": mabs,
        "sigmaint": 0.4,
        "alpha_low": alpha,
        "alpha_high": alpha,
        "beta": beta,
    }
)

# Apply realistic noise
errormodel = sim.noise_model
errormodel.pop("localcolor", None)
noisy_snia = snia.apply_gaussian_noise(errormodel)

# Create dataframe with only needed columns
df = noisy_snia.data

magobs = df["magobs"].to_numpy(dtype=np.float32)
x1     = df["x1"].to_numpy(dtype=np.float32)
c      = df["c"].to_numpy(dtype=np.float32)
z      = df["z"].to_numpy(dtype=np.float32)

# Reshape dataframe into a 1D numpy array containing the relevant data
flat = np.concatenate([magobs, x1, c, z])
flat_params = np.concatenate([[alpha, beta, mabs], flat]).astype(np.float32)

# Applying selection effect
print('Applying selection effect...')
m_lim = 19.5
simulation = se.malmquist_bias(flat_params, m_lim, N)
print('...done')


Applying selection effect...
...done


# Save to disk

In [ ]:
np.save("data/inference_data.npy", simulation)